In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
train_file = 'train_data.csv'
test_file = 'test_data.csv'

data_path = '/opt/ml/input/data/train_dataset'
train_path = os.path.join(data_path, train_file)
test_path = os.path.join(data_path, test_file)

In [ ]:
train_df = pd.read_csv(train_path, parse_dates=['Timestamp'])
test_df = pd.read_csv(test_path, parse_dates=['Timestamp'])

### 1) test set의 모든 userID가 train set에는 없다

In [ ]:
result = test_df['userID'].isin(train_df['userID'].unique())
all(~result)

### 2) test set의 assessmentItemID(=testId=KnowledgeTag) 는 모두 train set에 존재한다

In [ ]:
result = test_df['assessmentItemID'].isin(train_df['assessmentItemID'].unique())
print(all(result))
result = test_df['testId'].isin(train_df['testId'].unique())
print(all(result))
result = test_df['KnowledgeTag'].isin(train_df['KnowledgeTag'].unique())
print(all(result))

### 3) userID 마다 answerCode가 -1 인 row가 하나뿐이다. -1 은 마지막 timestamp를 갖는다

In [ ]:
unique_id = len(test_df['userID'].unique())
minus_len = len(test_df[test_df['answerCode'] == -1])
minus_unique_id = len(test_df[test_df['answerCode'] == -1]['userID'].unique())

print(unique_id, minus_len, minus_unique_id, )

### 4) test set내 unique 개수

In [ ]:
minus_df = test_df[test_df['answerCode'] == -1]

In [ ]:
unique_assessment = minus_df['assessmentItemID'].unique()
unique_test = minus_df['testId'].unique()
unique_tag = minus_df['KnowledgeTag'].unique()
print(unique_assessment.size)
print(unique_test.size)
print(unique_tag.size)

### 5) 마지막 문제와 같은 문제를 푼 기록이 있기도 하다

In [ ]:
minus_question = minus_df[['userID', 'assessmentItemID']].values

In [ ]:
for uid, itemid in minus_question:
    res = test_df[(test_df['userID'] == uid) & (test_df['assessmentItemID'] == itemid)]
    if len(res) > 1:
        print(uid, itemid)

In [ ]:
test_df[(test_df['userID'] == 203) & (test_df['assessmentItemID'] == 'A060073007')]

In [ ]:
train_df['tmp_index'] = train_df.index

In [ ]:
tmp_df = train_df[['userID', 'tmp_index', 'Timestamp']]

In [ ]:
tmp_df['tmp_index'] = tmp_df['tmp_index']+1

In [ ]:
train_elapse = train_df.merge(tmp_df, how='left', on=['userID', 'tmp_index'])

In [ ]:
train_elapse['elapse'] = train_elapse['Timestamp_x'] - train_elapse['Timestamp_y']

In [ ]:
train_elapse[train_elapse['assessmentItemID'] == 'A060073007']

### 6) 예측할때 해당 문제의 소요시간을 알 수 없다.
#### 사용할만한 것?
* 해당 문제 평균 소요시간
* 이전 문제 소요시간


In [ ]:
test_df['tmp_index'] = test_df.index

In [ ]:
tmp_df = test_df[['userID', 'tmp_index', 'Timestamp']]

In [ ]:
tmp_df.loc[:, 'tmp_index'] += 1

In [ ]:
test_elapse = test_df.merge(tmp_df, how='left', on=['userID', 'tmp_index'])

In [ ]:
test_elapse['elapse'] = test_elapse['Timestamp_x'] - test_elapse['Timestamp_y']

In [ ]:
five = timedelta(hours=5)

In [ ]:
test_elapse[test_elapse['elapse'] > five]

In [ ]:
## 유저별로 그룹할때 시험지 변화 찾기

In [ ]:
ggg = test_elapse.groupby(['userID'])

In [ ]:
duped_user_id = []
for key, gr in ggg:
    dup_testID = []
    prior_testID = gr.iloc[0]['testId']
    
    for index, row in gr.iterrows():
        if prior_testID == row['testId']:
            continue
        
        if row['testId'] in dup_testID:
            duped_user_id.append(row['userID'])
            break
        else:
            dup_testID.append(row['testId'])
            prior_testID = row['testId']


In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
ggg.get_group(duped_user_id[0])

In [ ]:
pd.set_option('display.max_rows', 10)

#### 풀다가 다음날 푸는 경우가 있다. (큰 시간차이)
#### 여러 시험지를 왔다갔다 하면서 푸는 경우가 있다. (시간을 어떻게 적용시켜 줄것? 바로 다음 시험문제를 푼다고 확신할 수 있는가? 필요가 있는가?)

In [ ]:
### 유저 + 시험지별 분류

In [ ]:
test_df['tmp_index'] = test_df.index

In [ ]:
tmp_df = test_df.loc[:, ['userID', 'testId', 'tmp_index', 'Timestamp']]

In [ ]:
tmp_df.loc[:, 'tmp_index'] -= 1

In [ ]:
test_elapse = test_df.merge(tmp_df, how='left', on=['userID', 'testId', 'tmp_index'])

In [ ]:
test_elapse['elapse'] = (test_elapse['Timestamp_y']  - test_elapse['Timestamp_x']).dt.seconds

In [ ]:
test_elapse

In [ ]:
test_elapse['elapse'] = test_elapse['elapse'].fillna(0).astype(int)

In [ ]:
test_elapse.loc[test_elapse['elapse'] > 600, 'elapse'] = 0

In [ ]:
test_elapse.elapse.value_counts()

### GroupShuffleSplit - 필요없음

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

In [ ]:
gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)

In [ ]:
for train, test in gss.split(train_df.index.values, groups=train_df.userID.values):
    print(train.size)
    print(test.size)

### LabelEncoder - 

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
assess = minus_df['assessmentItemID'].values
le.fit(assess)

In [ ]:
le.transform(['A080126008'])

### 시간별 count

In [ ]:
ttt = test_elapse.copy()

In [ ]:
ttt.isna().sum()

In [ ]:
ttt.count()

In [ ]:
42926 / (217188 + 42926)

In [ ]:
ttt_drop = ttt.dropna()

In [ ]:
ttt_drop.head(3)

In [ ]:
ttt_drop['elapse'] = (ttt_drop['elapse'].astype(np.int) / 10**9).astype(np.int)

In [ ]:
ttt_count = ttt_drop[['elapse', 'assessmentItemID']].groupby('elapse').count()

In [ ]:
# 카운트 10개 이상만 남긴다
ttt_above_10 = ttt_count[ttt_count['assessmentItemID'] > 10]

In [ ]:
ttt_above_10.index

In [ ]:
plt.plot(ttt_above_10.index, ttt_above_10.values, 'ro', markersize=1)

In [ ]:
plt.bar(ttt_above_10[:100].index.values, ttt_above_10[:100].assessmentItemID.values)

### 7) 대분류 추가

In [ ]:
minus_df['big_cate'] = minus_df['testId'].str[2]

### 8) 시험지별 -> 점수, 시간

### 9) 정답률 - 전체, 누적

### 10) 테스트 데이터 살펴보기

In [ ]:
row_count = test_df.groupby('userID')['userID'].count().values

In [ ]:
plt.plot(row_count, 'bo', markersize=2)
plt.axhline(row_count.mean(), color='red')
plt.title('averge_row')
plt.show()

In [ ]:
train_row_count = train_df.groupby('userID')['userID'].count().values
plt.plot(train_row_count, 'bo', markersize=2)
plt.axhline(train_row_count.mean(), color='red')
plt.title('averge_row')
plt.show()

In [ ]:
last_times = test_df[test_df['answerCode'] == -1].Timestamp

In [ ]:
last_times.min()

In [ ]:
last_times.max()

In [ ]:
train_df.groupby('userID').Timestamp.max().min()

In [ ]:
train_df.groupby('userID').Timestamp.max().max()

In [ ]:
t = train_df[['userID', 'Timestamp']].groupby(['userID']).agg({
    'userID':'count',
    'Timestamp':'max'
})

In [ ]:
t = t.sort_values('Timestamp')

In [ ]:
plt.plot(t['userID'].values)

In [ ]:
t = test_df[['userID', 'Timestamp']].groupby(['userID']).agg({
    'userID':'count',
    'Timestamp':'max'
})

In [ ]:
t = t.sort_values('Timestamp')

In [ ]:
t

In [ ]:
plt.plot(t['userID'].values)